In [58]:
import pandas as pd
from collocate_replacer import CollocateMatrix, PMI, Tokenizer
from collocate_replacer import most_distant_same_pos
from gensim.models import KeyedVectors
from tqdm import tqdm_notebook

In [2]:
df = pd.read_csv("unsupervised_news.csv")

In [3]:
tokenizer = Tokenizer(method='spacy_upos')

In [4]:
pmi = PMI()

In [5]:
cm = CollocateMatrix(3, tokenizer, pmi)

In [5]:
titles = df['title'].dropna()

In [6]:
is_str = lambda x: type(x) == str
titles = titles[titles.apply(is_str)==True].str.lower()

Нужно было всё привести к нижнему регистру, потому что модель uncased

In [59]:
cm.train(titles.values)

Training N-gram matrix...
Tokenizing...



Calculating n-gram frequencies...



Calculating metric...


In [12]:
cm.get_collocations(titles[0].lower())

[((2, 3), 'confirm_VERBas_SCONJ', 3.527744869294967),
 ((3, 4), 'as_SCONJeducation_NOUN', 3.360300388374646),
 ((4, 5), 'education_NOUNsecretary_NOUN', 6.899823330988027),
 ((9, 10), 'cast_VERBhistoric_ADJ', 5.46889179141474),
 ((10, 11), 'historic_ADJtie_NOUN', 4.580032714001256)]

In [11]:
titles[0]

'Betsy DeVos Confirmed as Education Secretary, With Pence Casting Historic Tie-Breaking Vote'

In [43]:
from ast import literal_eval

In [63]:
def save(self, directory):
    if not os.path.exists(directory):
        os.mkdir(directory)
    for key,val in self.__dict__.items():
        if type(val) == dict or type(val) == list:
            with open(os.path.join(directory, f"{key}.json"),'w',encoding='utf-8') as outp:
                if type(val) == dict:
                    json.dump({str(a):b for a,b in val.items()}, outp, ensure_ascii=False)
                elif type(val) == list:
                    json.dump(val, outp, ensure_ascii=False)
        elif type(val) == csr_matrix:
            save_npz(os.path.join(directory, f"{key}.npz"), val)

def load(self, directory):
        for file in os.listdir(directory):
            fname, ext = file.split('.')
            if ext == 'json':
                with open(os.path.join(directory, file), 'r', encoding='utf-8') as inp:
                    self.__setattr__(fname, json.load(inp))
                    if type(getattr(self,fname)) == dict:
                        d = getattr(self,fname)
                        for key in d:
                            if re.match("\([a-zA-Z0-9_]+, *?[a-zA-Z0-9_]+\)",key):
                                key_parsed = literal_eval(key)
                                d[key_parsed] = d[key]
                                d.pop(key)
                    elif type(getattr(self,fname)) == dict:
                        l = getattr(self,fname)
                        for i in range(len(l)):
                            if type(l[i]) == list:
                                l[i] = tuple(l[i])
            elif ext == 'npz':
                self.__setattr__(fname, load_npz(os.path.join(directory, file)))

In [60]:
cm.save(cm, 'CM_SpaCy')

In [7]:
cm1 = CollocateMatrix(3, tokenizer, pmi)
cm1.load("CM_SpaCy")

In [8]:
cm1.id2key[-100:]

[['’s_PART', 'hamilton_PROPN', 'outburst_PROPN'],
 ['the_DET', 'theater_NOUN', '’s_PART'],
 ['theater_NOUN', '’s_PART', 'history_NOUN'],
 ['’s_PART', 'history_NOUN', 'as_SCONJ'],
 ['’s_PROPN', 'win_NOUN', 'open_VERB'],
 ['ryan_PROPN', '’s_PART', 'vision_NOUN'],
 ['secretary_PROPN', 'pick_PROPN', 'be_AUX'],
 ['nancy_PROPN', 'pelosi_PROPN', '’s_PROPN'],
 ['pelosi_PROPN', '’s_PROPN', 'victory_NOUN'],
 ['’s_PROPN', 'victory_NOUN', 'show_VERB'],
 ['victory_NOUN', 'show_VERB', 'house_PROPN'],
 ['show_VERB', 'house_PROPN', 'democrats_PROPN'],
 ['house_PROPN', 'democrats_PROPN', 'do_AUX'],
 ['2016_NUM', '’s_PART', 'bizarre_ADJ'],
 ['’s_PART', 'bizarre_ADJ', 'presidential_ADJ'],
 ['trump_NOUN', '’s_PART', 'approach_NOUN'],
 ['hamilton_PROPN', 'mixtape_PROPN', '’s_PART'],
 ['mixtape_PROPN', '’s_PART', 'concert_NOUN'],
 ['’s_PART', 'concert_NOUN', 'live_PROPN'],
 ['that_DET', 'trump_PROPN', '’s_PART'],
 ['trump_PROPN', '’s_PART', 'idea_NOUN'],
 ['’s_PART', 'idea_NOUN', 'be_AUX'],
 ['of_ADP', 'jfk

In [27]:
list(key for key,val in cm.__dict__.items())

['n',
 'tokenizer',
 'N',
 'counts',
 'ngram_matrix',
 'id2key',
 'key2id',
 'metric',
 'collocate_matrix']

In [10]:
cm1.get_collocations(titles[0], thresh=0)

[((0, 1), ('betsy_PROPN', 'devos_PROPN'), 9.298342026997148),
 ((1, 2), ('devos_PROPN', 'confirm_VERB'), 5.5508847123890375),
 ((2, 3), ('confirm_VERB', 'as_SCONJ'), 3.2203468732129714),
 ((3, 4), ('as_SCONJ', 'education_NOUN'), 3.143938027065899),
 ((4, 5), ('education_NOUN', 'secretary_NOUN'), 6.395061522399001),
 ((5, 6), ('secretary_NOUN', ',_PUNCT'), 0.9008710183501698),
 ((7, 8), ('with_ADP', 'penny_NOUN'), 1.491382998988028),
 ((8, 9), ('penny_NOUN', 'cast_VERB'), 3.682011797848684),
 ((9, 10), ('cast_VERB', 'historic_ADJ'), 4.983888817155992),
 ((10, 11), ('historic_ADJ', 'tie_NOUN'), 4.387803349439177),
 ((11, 12), ('tie_NOUN', '-_PUNCT'), 1.0584752321335404),
 ((12, 13), ('-_PUNCT', 'break_VERB'), 0.503662962084169),
 ((13, 14), ('break_VERB', 'vote_NOUN'), 1.5859149676470743)]

In [12]:
[i for i in cm1.key2id if type(i)==tuple]

[]

In [16]:
wv_model = KeyedVectors.load_word2vec_format("../gensim_models/udpipe_wikipedia/model.bin",
                                            binary = True)

In [32]:
most_distant_same_pos('obama_PROPN', model=wv_model, thresh=0.5)

KeyError: "word 'obama_PROPN' not in vocabulary"

In [29]:
wv_model['Trump_PROPN']

array([ 0.33118704, -0.02581371,  0.5585557 , -0.07298037,  0.08278263,
       -0.3434537 , -0.4469778 , -0.5176432 ,  0.08702073, -0.06692564,
        0.05944035,  0.4230861 , -0.0442996 ,  0.09940568,  0.05434435,
       -0.08007045,  0.1581048 ,  0.11037294,  0.03450404,  0.04754427,
        0.17007348,  0.13349432,  0.32757118, -0.11025832, -0.17576501,
       -0.45155552,  0.04854733, -0.22571519, -0.09868459,  0.00647056,
        0.2407091 ,  0.04586773, -0.20840503, -0.04097489, -0.0228256 ,
       -0.19320111,  0.04269384, -0.1918266 ,  0.10792869,  0.1115344 ,
       -0.02387644,  0.3142858 ,  0.2620476 , -0.40638   , -0.5472039 ,
        0.2425795 ,  0.23102237,  0.13447256, -0.18926024,  0.39277232,
        0.23044646, -0.03210957,  0.07349224, -0.20458417, -0.62585527,
        0.00930427,  0.09734678,  0.37851995, -0.36719686, -0.1099631 ,
       -0.1499483 , -0.03544396, -0.00815317, -0.16387387,  0.27113897,
        0.09094331, -0.2983506 , -0.05958993,  0.18857451, -0.15

In [73]:
def replace_collocates(s, cm, wv_model, dist_thresh=0.65, colloc_thresh=2.0):
    collocations = cm.get_collocations(s, thresh=colloc_thresh)
    collocations = sorted(collocations, key=lambda x: x[2], reverse=False)
    output = []
    
    for colloc in collocations:
        word_a, word_b = colloc[1]
        
        if not word_a.split('_')[-1] in ('PART','CCONJ', 'SCONJ', 'ADP') and\
        not word_b.split('_')[-1] in ('PART','CCONJ', 'SCONJ', 'ADP'):
            pass
        else:
            continue
        
        try:
            candidates = most_distant_same_pos(word_b, wv_model, thresh=dist_thresh)
        except KeyError:
            continue
        for candidate, candidate_dist in candidates:
            strength = cm.collocation_strength(word_a, candidate)
            if strength is not None:
                if strength > colloc_thresh:
                    output.append((colloc[0], (word_a, word_b, colloc[2]), (word_a, candidate,strength),
                                  candidate_dist))
        
        ## В другом порядке:
        word_b, word_a = word_a, word_b
        
        try:
            candidates = most_distant_same_pos(word_b, wv_model, thresh=dist_thresh)
        except KeyError:
            continue
        for candidate, candidate_dist in candidates:
            strength = cm.collocation_strength(word_a, candidate)
            if strength is not None:
                if strength > colloc_thresh:
                    output.append((colloc[0], (word_a, word_b, colloc[2]), (word_a, candidate,strength),
                                  candidate_dist))
    return output

In [74]:
output = []

for title in tqdm_notebook(titles[:1000], total=1000):
    unit = replace_collocates(title, cm1, wv_model, dist_thresh=0.55)
    if unit:
        output.append((title, unit))

c:\users\k1l77\desktop\182e~1\term_p~1\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [69]:
cm1.collocation_strength('prime_ADJ','minister_NOUN')

8.193875694606945

In [75]:
len(output) == 0

False

In [76]:
len(output)

274

In [77]:
output

[('melania trump says white house could mean millions for brand',
  [((5, 6),
    ('could_VERB', 'mean_VERB', 3.985150443992131),
    ('could_VERB', 'resurface_VERB', 4.485685376176988),
    0.5693224668502808)]),
 ('multi-state manhunt in southeast intensifies for alleged murderer and accomplice',
  [((1, 2),
    ('-_ADJ', 'state_ADJ', 5.594271722145067),
    ('-_ADJ', 'natal_ADJ', 6.6928840108131755),
    0.5937535166740417),
   ((1, 2),
    ('-_ADJ', 'state_ADJ', 5.594271722145067),
    ('-_ADJ', 'estrogen_ADJ', 6.6928840108131755),
    0.5629760026931763)]),
 ('flu takes a toll in nyc with 4 children reported dead in seasonal outbreak',
  [((0, 1),
    ('take_VERB', 'flu_NOUN', 2.299350973038946),
    ('take_VERB', 'ax_NOUN', 4.15564896340457),
    0.5779735445976257),
   ((0, 1),
    ('take_VERB', 'flu_NOUN', 2.299350973038946),
    ('take_VERB', 'stand_NOUN', 3.189933348202893),
    0.5525599718093872)]),
 ("dept. of justice cites 'national security' in calling for trump immigrat